In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from shutil import copyfile
from tqdm import tqdm

In [2]:
train = pd.read_csv('../input/tensorflow-great-barrier-reef/train.csv')
train['pos'] = train.annotations != '[]'

In [3]:
train.head()

,video_id,sequence,video_frame,sequence_frame,image_id,annotations,pos
0,0,40258,0,0,0-0,[],False
1,0,40258,1,1,0-1,[],False
2,0,40258,2,2,0-2,[],False
3,0,40258,3,3,0-3,[],False
4,0,40258,4,4,0-4,[],False


In [4]:
!mkdir -p ./yolo_data/fold1/images/val
!mkdir -p ./yolo_data/fold1/images/train

!mkdir -p ./yolo_data/fold1/labels/val
!mkdir -p ./yolo_data/fold1/labels/train

In [5]:
fold = 1

annos = []
for i, x in tqdm(train.iterrows()):
#     print(i,x)
#     break
    if x.video_id == fold:
        mode = 'val'
    else:
        # train
        mode = 'train'
        if not x.pos: continue
        # val
    copyfile(f'../input/tensorflow-great-barrier-reef/train_images/video_{x.video_id}/{x.video_frame}.jpg',
                f'./yolo_data/fold{fold}/images/{mode}/{x.image_id}.jpg')
    if not x.pos:
        continue
    r = ''
    anno = eval(x.annotations)
    for an in anno:
        r += '0 {} {} {} {}\n'.format((an['x'] + an['width'] / 2) / 1280,
                                        (an['y'] + an['height'] / 2) / 720,
                                        an['width'] / 1280, an['height'] / 720)
#         print(r)
#         print()
    with open(f'./yolo_data/fold{fold}/labels/{mode}/{x.image_id}.txt', 'w') as fp:
        fp.write(r)

23501it [01:58, 198.30it/s] 


In [6]:
train["video_id"].value_counts()

2    8561
1    8232
0    6708
Name: video_id, dtype: int64

In [7]:
import os
len(os.listdir("./yolo_data/fold1/images/train"))

2820

In [8]:
hyps = '''
# YOLOv5 Hyperparameter Evolution Results
# Best generation: 4
# Last generation: 5
#    metrics/precision,       metrics/recall,      metrics/mAP_0.5, metrics/mAP_0.5:0.95,         val/box_loss,         val/obj_loss,         val/cls_loss
#              0.16318,              0.11858,             0.064654,             0.023477,             0.039919,            0.0055961,                    0

lr0: 0.01048
lrf: 0.11894
momentum: 0.92909
weight_decay: 0.00052
warmup_epochs: 3.88107
warmup_momentum: 0.57065
warmup_bias_lr: 0.10278
box: 0.06726
cls: 0.46752
cls_pw: 1.44082
obj: 1.0107
obj_pw: 0.803
iou_t: 0.2
anchor_t: 3.1544
fl_gamma: 0.0
hsv_h: 0.01506
hsv_s: 0.79959
hsv_v: 0.49056
degrees: 0.0
translate: 0.09677
scale: 0.48172
shear: 0.0
perspective: 0.0
flipud: 0.0
fliplr: 0.5
mosaic: 0.95352
mixup: 0.0
copy_paste: 0.0
anchors: 2.55656
'''

In [9]:
data = '''
# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: ../yolo_data/fold1/  # dataset root dir
train: images/train  # train images (relative to 'path') 128 images
val: images/val  # val images (relative to 'path') 128 images
test:  # test images (optional)

# Classes
nc: 1  # number of classes
names: ['reef']  # class names


# Download script/URL (optional)
# download: https://ultralytics.com/assets/coco128.zip
'''

In [10]:
# !git clone https://github.com/ultralytics/yolov5.git
!git clone https://github.com/soumya997/yolov5-w-f2-mod.git
!mv ./yolov5-w-f2-mod ./yolov5

Cloning into 'yolov5-w-f2-mod'...
remote: Enumerating objects: 112, done.
remote: Counting objects: 100% (112/112), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 112 (delta 23), reused 105 (delta 19), pack-reused 0
Receiving objects: 100% (112/112), 1.38 MiB | 3.35 MiB/s, done.
Resolving deltas: 100% (23/23), done.


In [12]:
# !touch ./data/reef_f1_naive.yaml
# !touch ./data/hyps/hyp.heavy.2.yaml

In [13]:
# !pwd

In [14]:
with open('./yolov5/data/reef_f1_naive.yaml', 'w') as fp:
    fp.write(data)
with open('./yolov5/data/hyps/hyp.heavy.2.yaml', 'w') as fp:
    fp.write(hyps)

In [15]:
%cd yolov5

/kaggle/working/yolov5


In [16]:
!ls data

Argoverse.yaml	      VOC.yaml	     hyps		 xView.yaml
GlobalWheat2020.yaml  VisDrone.yaml  images
Objects365.yaml       coco.yaml      reef_f1_naive.yaml
SKU-110K.yaml	      coco128.yaml   scripts


In [17]:
!python train.py --img 1920 --batch 2 --epochs 7 --data reef_f1_naive.yaml --weights yolov5s6.pt --name l6_3600_uflip_vm5_f1 --hyp data/hyps/hyp.heavy.2.yaml

wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: (30 second timeout) 
wandb: W&B disabled due to login timeout.
Weights & Biases: run 'pip install wandb' to automatically track and visualize YOLOv5 🚀 runs (RECOMMENDED)
100%|███████████████████████████████████████| 24.5M/24.5M [00:00<00:00, 220MB/s]

train: Scanning '../yolo_data/fold1/labels/train' images and labels...2820 found
val: Scanning '../yolo_data/fold1/labels/val' images and labels...2099 found, 61
AutoAnchor: Evolving anchors with Genetic Algorithm: fitness = 0.9003: 100%|█| 1
       0/6     4.18G   0.09725   0.04898         0         2      1920: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
       1/6     4.18G   0.06998   0.03293         0         2      1920: 100%|███
               Class     Images     Labels          P          R     mAP@.5 mAP@
       2/6     4.18G   0.05492   0.02688      

In [18]:
# !python train.py --img 1920\ 
# --batch 2\ 
# --epochs 7\ 
# --data reef_f1_naive.yaml\ 
# --weights yolov5s6.pt\
# --name base_yolo_5\ 
# --hyp data/hyps/hyp.heavy.2.yaml

In [19]:
# asdlkasnhdljnalsdnlaksndlkasnd hvkvk khvhkv  apijdpajsd apsjdpa asnpduhy7hy7hyn asdnpa apsjdpa asnpdn asdnpa iygiyugb giugu

# Kaggle dataset API call:

In [20]:
!rm /root/.kaggle

rm: cannot remove '/root/.kaggle': No such file or directory


In [21]:
!mkdir /root/.kaggle

In [22]:
!cp -v /kaggle/input/random-private-files/kaggle.json /root/.kaggle

'/kaggle/input/random-private-files/kaggle.json' -> '/root/.kaggle/kaggle.json'


In [23]:
!kaggle datasets init -p ./runs/train/base_yolo_5/weights

Invalid folder: ./runs/train/base_yolo_5/weights


In [24]:
%%writefile ./runs/train/base_yolo_5/weights/dataset-metadata.json

{
  "title": "yolov5_train_2_weights",
  "id": "soumya9977/yolov5-train-2-weights",
  "licenses": [
    {
      "name": "CC0-1.0"
    }
  ]
}

Writing ./runs/train/base_yolo_5/weights/dataset-metadata.json


FileNotFoundError: [Errno 2] No such file or directory: './runs/train/base_yolo_5/weights/dataset-metadata.json'

In [ ]:
!kaggle datasets create -p ./runs/train/base_yolo_5/weights

In [ ]:
# !kaggle datasets version -p ./runs/train/l6_3600_uflip_vm5_f1/weights -m "adding model weights"

In [ ]:
# !cp -v ./runs/train/l6_3600_uflip_vm5_f1/dataset-metadata.json ./runs/train/l6_3600_uflip_vm5_f1/weights

In [ ]:
!ls -GFlash --color .

# copying to the working:

In [ ]:
!cp -R ./runs/train/base_yolo_5 /kaggle/working